# Selenium Scraping

In [114]:
import pandas as pd 
import re
from selenium import webdriver
import time

# Twitter Scrapping using selenium

In [18]:


SCROLL_PAUSE_TIME = 15
tweets = []

#for word in search_words:
word = "Donald Trump"
#words.append(word)
driver = webdriver.Chrome(executable_path='chromedriver.exe')
path = "https://twitter.com/search?q=Donald%20Trump%20min_replies%3A100%20min_faves%3A100%20min_retweets%3A100%20lang%3Aen%20until%3A2020-11-28%20since%3A2020-01-01&src=typed_query&f=live"
driver.get(path)
time.sleep(10)
#tweets=[]
data = driver.find_elements_by_xpath("//div[@aria-label='Timeline: Search timeline']")
last_height = driver.execute_script("return document.body.scrollHeight")

while True:
    #time.sleep(5)
    # Scroll down to bottom
    driver.execute_script("window.scrollTo(0, document.body.scrollHeight);")

    # Wait to load page
    time.sleep(SCROLL_PAUSE_TIME)

    data = driver.find_elements_by_xpath("//div[@aria-label='Timeline: Search timeline']")

    for d in data:
        tweets.append(d.text)
        #print(d.text)

    # Calculate new scroll height and compare with last scroll height
    new_height = driver.execute_script("return document.body.scrollHeight")
    if new_height == last_height:
        break
    last_height = new_height
    #break



In [22]:
tweets = pd.DataFrame(tweets)
tweets.to_csv("Tweets.csv")

In [190]:
data = pd.read_csv("Tweets.csv")
data.head()

,Unnamed: 0,0
0,0,Don Winslow\n@donwinslow\n·\n10h\nDear \n@real...
1,1,Ronna McDaniel\n@GOPChairwoman\n·\n13h\nEnthus...
2,2,MURRAY\n@murray_nyc\n·\n14h\nReplying to \n@re...
3,3,Glenn Kirschner\n@glennkirschner2\n·\n15h\nDon...
4,4,Jeff Tiedrich\n@itsJeffTiedrich\n·\n15h\nReply...


In [206]:
text = data.iloc[:,1]
text

0       Don Winslow\n@donwinslow\n·\n10h\nDear \n@real...
1       Ronna McDaniel\n@GOPChairwoman\n·\n13h\nEnthus...
2       MURRAY\n@murray_nyc\n·\n14h\nReplying to \n@re...
3       Glenn Kirschner\n@glennkirschner2\n·\n15h\nDon...
4       Jeff Tiedrich\n@itsJeffTiedrich\n·\n15h\nReply...
                              ...                        
1347    ESPN\n@espn\n·\nJun 6\nSaints players voiced t...
1348    Drew Brees\n@drewbrees\n·\nJun 5\nTo \n@realdo...
1349    AprilDRyan\n@AprilDRyan\n·\nJun 5\nThis is way...
1350    DJ Judd\n@DJJudd\n·\nJun 5\nThat’s the stars a...
1351    Holly Figueroa O'Reilly\n@AynRandPaulRyan\n·\n...
Name: 0, Length: 1352, dtype: object

In [207]:
tweets_lst = []
author = []
date = []
likes = []
retweets = []
comments = []
for tweet in text:
    #print(tweet)
    t = re.split("(\@[A-z0-9_]+\\n·)",tweet)
    t = t[1:]
    for i in range(len(t)):
        
        if i % 2 == 0:
            auth = re.split("\\n·",t[i])[0]
            author.append(auth)
        else:
            txt = re.split("\\n",t[i])
            txt = txt[1:]
            date.append(txt[0])
            if not (re.match("\ASome or all of the content shared in this Tweet",txt[1]) or re.match("\AThis Tweet violated the Twitter Rules about abusive behavior",txt[1]) ):
                #print(txt)
                if txt[-1] == "":
                    txt = txt[:-2]
                while not (re.match("\A[0-9]",txt[-1])):
                    txt = txt[:-1]
                #print(txt)
                likes.append(txt[-1])
                retweets.append(txt[-2])
                comments.append(txt[-3])
                twt = " ".join(txt[1:-3])
                tweets_lst.append(twt)
            else:
                likes.append(" ")
                retweets.append(" ")
                comments.append(" ")
                tweets_lst.append(" ")
    #break

In [208]:
len(likes)

17586

In [209]:
#df = [author, date, tweets_lst, likes, comments, retweets]
tweets_df = pd.DataFrame({'Author' : author,
                          "Date" : date,
                          'Tweet' : tweets_lst,
                          'Likes' : likes,
                          'Comments' : comments,
                          'Retweets' : retweets})
tweets_df.head()

,Author,Date,Tweet,Likes,Comments,Retweets
0,@donwinslow,10h,Dear @realDonaldTrump President-elect Biden's...,15.8K,173,2.3K
1,@Strandjunker,10h,"To heal, we need Nuremberg Trials 2.0 against ...",15K,415,3K
2,@tonyposnanski,10h,There is no reason why Donald Trump shouldn’t ...,5.5K,167,550
3,@DrSamGirgis,11h,Watch as 20K PA votes for Trump disappear in r...,17.9K,1.1K,11.2K
4,@mmpadellan,11h,I don't care if trump voluntarily tries to lea...,11.3K,321,1.3K


In [210]:
mentions = []
for text in tweets_df['Tweet']:
    t = re.split(" ",text)
    mention = [m for m in t if m.startswith("@")]
    mentions.append(", ".join(mention))

In [211]:
tweets_df['Mentions'] = pd.DataFrame(mentions,columns=['Mentions'])
tweets_df.head(10)

,Author,Date,Tweet,Likes,Comments,Retweets,Mentions
0,@donwinslow,10h,Dear @realDonaldTrump President-elect Biden's...,15.8K,173,2.3K,@realDonaldTrump
1,@Strandjunker,10h,"To heal, we need Nuremberg Trials 2.0 against ...",15K,415,3K,
2,@tonyposnanski,10h,There is no reason why Donald Trump shouldn’t ...,5.5K,167,550,
3,@DrSamGirgis,11h,Watch as 20K PA votes for Trump disappear in r...,17.9K,1.1K,11.2K,"@CNN, @realDonaldTrump, @JoeBiden, @JohnKingCNN"
4,@mmpadellan,11h,I don't care if trump voluntarily tries to lea...,11.3K,321,1.3K,
5,@murray_nyc,11h,Replying to @realDonaldTrump This is fake new...,9.6K,281,480,@realDonaldTrump
6,@JennaEllisEsq,11h,Arizona State Legislature to hold hearing on e...,88.8K,8.6K,25K,"@RudyGiuliani, @realDonaldTrump"
7,@Perduesenate,11h,"US Senate candidate, GA I’m proud to have @re...",780,255,163,"@realDonaldTrump, @kloeffler"
8,@sahilkapur,11h,Joe Biden handily won Americans who make under...,16K,429,3.2K,@RadioFreeTom
9,@itsJeffTiedrich,11h,"Replying to @realDonaldTrump you lost, butter...",12K,408,421,@realDonaldTrump


In [215]:
tweets_df = tweets_df[tweets_df['Tweet'] != " "]
tweets_df.shape

(17583, 7)

In [216]:
tweets_df.to_csv("Tweet Information new.csv")

# Data Cleaning

In [93]:
data = pd.read_csv("Tweet Information.csv")
data.head()

,Author,Date,Tweet,Likes,Comments,Retweets,Mentions
0,@donwinslow,28-Nov,Dear @realDonaldTrump President-elect Biden's...,15.8K,173,2.3K,@realDonaldTrump
1,@Strandjunker,28-Nov,"To heal, we need Nuremberg Trials 2.0 against ...",15K,415,3K,NaN
2,@tonyposnanski,28-Nov,There is no reason why Donald Trump shouldn’t ...,5.5K,167,550,NaN
3,@DrSamGirgis,28-Nov,Watch as 20K PA votes for Trump disappear in r...,17.9K,1.1K,11.2K,"@CNN, @realDonaldTrump, @JoeBiden, @JohnKingCNN"
4,@mmpadellan,28-Nov,I don't care if trump voluntarily tries to lea...,11.3K,321,1.3K,NaN


In [94]:
import tweepy

consumer_key = "Qw1qwbCktYuQ5kNrfNY3dNZBp"
consumer_secret = "NfKRseB2TiwK8pPdM0Rl8w5OgDKOgkAjJ9p7vghWagJMFlcoln"
access_token = "452312788-psGeiOb0gjonibDJkxd36ny8KDVQt8biEQYUZ4iR"
access_token_secret = "saHiKiM22ZWLmAarOfOZvxOgU6hWnF1hgZfPmVnLj7gJ9"

auth = tweepy.OAuthHandler(consumer_key, consumer_secret)
auth.set_access_token(access_token, access_token_secret)
api = tweepy.API(auth,wait_on_rate_limit=True)

In [105]:
tweet_list = []
i = 0
for row in data['Author'].unique():
    print(i)
    print(row)
    try:
        tweet = api.get_user(screen_name=row)
        tweet_list.append([tweet.id,tweet.name,tweet.screen_name,tweet.location,tweet.followers_count,tweet.friends_count,
                     tweet.listed_count,tweet.created_at,tweet.favourites_count,tweet.verified,tweet.statuses_count])        
    except Exception as err:
        print(err)
    i = i + 1

0
@donwinslow
1
@Strandjunker
2
@tonyposnanski
3
@DrSamGirgis
4
@mmpadellan
5
@murray_nyc
6
@JennaEllisEsq
7
@Perduesenate
8
@sahilkapur
9
@itsJeffTiedrich
10
@mike_wanaoni
11
@realDonaldTrump
12
@GOPChairwoman
13
@Breaking911
14
@gtconway3d
15
@prchovanec
16
@ScottPresler
17
@NotHoodlum
18
@thebr0keb0i
19
@SRuhle
20
@KLoeffler
21
@wolfblitzer
22
@TalbertSwan
23
@TheRightMelissa
24
@gregkellyusa
25
@TomFitton
26
@glennkirschner2
27
@CaslerNoel
28
@newsmax
29
@BrandonStraka
30
@Used_To_Be_GOP
31
@Barnes_Law
32
@TheOfficerTatum
33
@InfidelAngela
34
@MatthewJshow
35
@kimguilfoyle
36
@kirkacevedo
37
@ScottFishman
38
@SaraCarterDC
39
@Hoosiers1986
40
@tommyinnit
41
@JackPosobiec
42
@ChuckCallesto
43
@drdavidsamadi
44
@Franklin_Graham
45
@joshtpm
46
@LaurenWitzkeDE
47
@brianklaas
48
@JamesKosur
49
@GenFlynn
50
@piersmorgan
51
@CNN
52
@SkyNewsAust
53
@TheDemCoalition
54
@MarkDice
55
@DavMicRot
56
@Quackity
57
@honkkarl
58
@KarlJacobs_
59
@JoeTalkShow
60
@roper_93
[{'code': 50, 'message': 'Use

483
@TheDailyShow
484
@JazzShaw
485
@KeithOlbermann
486
@charlielangton
487
@DailyLimbaugh
488
@kayleighmcenany
489
@RandyRainbow
490
@ashishkjha
491
@LarrySabato
492
@MichaelRapaport
493
@David_Leavitt
494
@DannyZuker
495
@Phil_Mattingly
496
@ProjectLincoln
497
@APFactCheck
498
@philiptirino
499
@Peoples_Pundit
500
@ericmetaxas
501
@alanwrylands
502
@AVindman
503
@justin_hart
504
@Fahrenthold
505
@insidernews
506
@THeinrich22
507
@RickyRebelRocks
508
@G_TheOriginal
509
@LizWillis_
510
@petersagal
511
@carrieksada
512
@JDRucker
513
@MattOswaltVA
514
@Scaramucci
515
@DrOlufunmilayo
516
@OxfordDiplomat
517
@TomiLahren
518
@JohnFBachman
519
@davidkurten
520
@JustVent6
521
@PeterAHarder
522
@NewDay
523
@Craig_A_Spencer
524
@bessbell
525
@Annakhait
526
@MichaelT_45
527
@beyouonlybetter
528
@rolandsmartin
529
@nickveniamin
530
@ChrisRuddyNMX
531
@tomselliott
532
@RealKyleMorris
533
@StopTheCriminal
534
@mdx3232
535
@petestrzok
536
@Mocraig13
537
@GrahamAllen_1
538
@SandySmithNC
539
@sfpelosi

960
@DeborahMeaden
961
@TonyBellew
962
@ItsAndyRyan
963
@guyverhofstadt
964
@donlemon
965
@christina_bobb
966
@bluestein
967
@Cleavon_MD
968
@LilNasX
969
@mrjamesob
970
@nwarikoo
971
@RealMadDadMAGA
972
@brianstelter
973
@colbertlateshow
974
@rooshv
975
@DiamondandSilk
976
@Complex
977
@JasonKander
978
@RepMarkWalker
979
@SBSNews
980
@JoeBiden
981
@The_Real_DiMaK
982
@LionelMedia
983
@ShaneGoldmacher
984
@dougmillsnyt
985
@yashar
986
@larsloekke
987
@DanPriceSeattle
988
@WindsorMann
989
@chrisbergpov
990
@mariashriver
991
@bobhardt
992
@EliseStefanik
993
@bonnievanasse
994
@svdate
995
@kellidrum
996
@ClayTravis
997
@SenJohnBarrasso
998
@SenatorBraun
999
@scrowder
1000
@RyannMcEnany
1001
@RepAlexMooney
1002
@TommyHicksGOP
1003
@HeyTammyBruce
1004
@TrumpWarRoom
1005
@FrankLuntz
1006
@KatrinaPierson
1007
@TimMurtaugh
1008
@SenateGOP
1009
@arielhelwani
1010
@KellyO
1011
@TrumpVoterWTF
1012
@JimGaffigan
1013
@LarrySchweikart
1014
@Channel4News
1015
@SusanPage
1016
@AOECOIN
1017
@davemacladd

1414
@thomaskaine5
1415
@Bill_Maxwell_
1416
@TG22110
1417
@ATX_PATRIOT
1418
@nytopinion
1419
@Beverly79943415
1420
@kathrynresister
1421
@marklutchman
1422
@DevinCow
1423
@SharriMarkson
1424
@SenCoryGardner
1425
@bellamaria1776
1426
@Leslieoo7
1427
@angela214
1428
@TroyE23
1429
@kelly_neidert
1430
@ReportsDaNews
1431
@GarrettHaake
1432
@maxasteele
1433
@VBermoothes
1434
@MollyBeck
1435
@timallanwalker
1436
@kayrosef
1437
@soledadobrien
1438
@LindaofNM1
1439
@RealBasedMAGA
1440
@mercedesschlapp
1441
@LucyPesk
1442
@Joy_Villa
1443
@carolecadwalla
1444
@JamesWorley
1445
@LinkedHD
1446
@TrumpGirlOnFire
1447
@chefjclark
1448
@AlyssaNo_L
1449
@RealDLHughley
1450
@GavinNewsom
1451
@HizbkKhan
1452
@NathanWurtzel
1453
@MattFinnFNC
1454
@JennPellegrino
1455
@kymn591
1456
@HoustonChron
1457
@DanAlexander21
1458
@HotPockets4All
1459
@BlueHenArt
1460
@JayCuse96
1461
@BCAppelbaum
1462
@mpollandsr
1463
@real_defender
1464
@jfcarraway
1465
@kealoha20
1466
@Styx666Official
1467
@TLee_is_me
1468
@aginnt

1863
@EddieZipperer
1864
@whatgives1313
1865
@GaryLineker
1866
@brucegreene
1867
@donaldtuskEPP
1868
@realDSteinberg
1869
@iam_danalena
1870
@AdamBandt
1871
@NYinLA2121
1872
@bradleyrsimpson
1873
@Bros4Joe
1874
@_cingraham
1875
@KendalKillian
1876
@julie_kelly2
1877
@baseballcrank
1878
@robynkanner
1879
@politicususa
1880
@katieporteroc
1881
@Out5p0ken
1882
@CBCNews
1883
@Mikel_Jollett
1884
@megc1977
1885
@Harry1T6
1886
@realDylanQuatt
1887
@maggieNYT
1888
@eSalonSF
1889
@kachelme1
1890
@CNNSotu
1891
@GovRidge
1892
@Jacfam2
1893
@Jacq4Peace
1894
@BillyRay5229
1895
@KenCook_C
1896
@SteveDaines
1897
@GovernorLittle
1898
@gtconway3dg
1899
@pickupapainting
1900
@Pattihatestrump
1901
@ShimonPro
1902
@LarryOConnor
1903
@BadBrad_1986
1904
@GAntalics
1905
@BillyBaldwin
1906
@lebeausrogue
1907
@MevlutCavusoglu
1908
@Rschooley
1909
@frankdpi
1910
@AuthorKimberley
1911
@Variety
1912
@rcn400
1913
@Ironhorse76
1914
@RalphHise
1915
@erichards24
1916
@GrassleyPress
1917
@MDPAnews
1918
@SunshineDreame

2316
@Dr_Dharma_Bum
2317
@WayTooEarly
2318
@grandmagrit
2319
@ruffLILPUP
2320
@thechrisbuskirk
2321
@Teesa_Michele
2322
@elizabethjdias
2323
@LASDBrink
2324
@thetoyman1
2325
@LisaS4680
2326
@PhilMcCrackin44
2327
@AgentSoulful007
2328
@ByronDonalds
2329
@RealBrysonGray
2330
@Ashleysdiary1
2331
@Jelly_Narwhal
2332
@blu_flag
2333
@ABC7NY
2334
@HOME_nola
2335
@TaylorPopielarz
2336
@owillis
2337
@ItsAboutTime45
2338
@BibatheDiva
2339
@notkristiane
2340
@_TheRealBecky1
2341
@Rachel__Nichols
2342
@CNNPR
2343
@BoKnowsNews
2344
@brikeilarcnn
2345
@RogerMarshallMD
2346
@VicBergerIV
2347
@jeremycorbyn
2348
@ChelseaClinton
2349
@normanswan
2350
@GregAbbott_TX
2351
@swingleft
2352
@Rasmussen_Poll
2353
@AmyFeller
2354
@MerileeParis
2355
@yournameherre
2356
@JeffTarlo
2357
@TheLaurenChen
2358
@NBSaphierMD
2359
@CalltoActivism
2360
@MyndiB
2361
@WileELiberal
2362
@PacificReports
2363
@SamanthaJPower
2364
@JoanneWT09
2365
@lynnluvseddie
2366
@jdlbrooklyn
2367
@JonahDispatch
2368
@HuffmanForNC
2369
@Sha

2763
@JohnMiehoff22
2764
@StefSimanowitz
2765
@JacquieLeyns
2766
@sondro406
2767
@14Truth14
2768
@truth_momma
2769
@FabioQanon
2770
@HollyDolly1977
2771
@LuvMacyIsabella
2772
@DFBHarvard
2773
@laralogan
2774
@Ylee1972
2775
@ChatChristi
2776
@BriefcaseMike
2777
@sixthsealnews
2778
@gswchris
2779
@heydi_geroy
2780
@texasdemocrats
2781
@SaltyDuchess
2782
@45defender
2783
@RB_Scott_80
2784
@KaraCalavera
2785
@joedunn721
2786
@Alice_Weidel
2787
@HackneyAbbott
2788
@espn
2789
@AlbertBreer
2790
@ThomasGervais8
2791
@drewbrees
2792
@Dawnitis


In [107]:
len(data['Author'].unique())

2793

In [108]:
len(tweet_list)

2791

In [109]:
author_df = pd.DataFrame(tweet_list, columns=['Id','Name','Screen Name','Location','Followers Count','Friends Count',
                                            'Listed Count','Created At','Favourites Count','Verified','Tweets Count'])

In [110]:
author_df.to_csv("Author Information.csv")

In [112]:
data[data['Author']=="@roper_93"]
#account deleted

,Author,Date,Tweet,Likes,Comments,Retweets,Mentions
93,@roper_93,26-Nov,Still can't believe that Donald Trump got 73 m...,6.2K,779,545,NaN
1503,@roper_93,15-Nov,I don't understand how these Republicans call ...,2.1K,197,357,NaN
2476,@roper_93,7-Nov,"Other than donald trump, who are you most exci...",1.8K,2.8K,415,NaN
8348,@roper_93,24-Sep,"Jesus H. Christ, we are literally here having ...",2.7K,329,611,NaN
11674,@roper_93,18-Aug,"So, Donald Trump lied to Robert Mueller and th...",15.5K,1.2K,4.2K,NaN
14053,@roper_93,18-Jul,I'm not bothered by Donald Trump's silence on ...,4.1K,289,719,NaN
15318,@roper_93,2-Jul,Whatever Vladimir Putin has on Donald Trump mu...,2K,235,691,NaN


In [113]:
didnt_askg
data[data['Author']=="@didnt_askg"]
#account deleted

NameError: name 'didnt_askg' is not defined